In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%load_ext autotime

In [ ]:
!rmdir /s /q images

In [ ]:
!mkdir images

In [ ]:
CATEGORIES = ['sunflower', 'rugby ball leather', 'ice cream cone']

In [ ]:
from bing_image_downloader import downloader

for category in CATEGORIES:
  downloader.download(category, limit=30, output_dir='images',
                    adult_filter_off=True, verbose=False)

In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import cv2

target = []
images = []
flat_data = []

DATADIR = "images"

for category in CATEGORIES:
    class_num = CATEGORIES.index(category)  # Label encoding the values
    path = os.path.join(DATADIR, category)  # Create path to use all the images

    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img))
        img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)  # OpenCV loads images in BGR format, convert to RGB
        # print(img_array.shape)
        # plt.imshow(img_array)
        img_resized = cv2.resize(img_array, (150, 150))  # Normalizes the value from 0 to 1
        flat_data.append(img_resized.flatten())
        images.append(img_resized)
        target.append(class_num)

flat_data = np.array(flat_data)
target = np.array(target)
images = np.array(images)

In [ ]:
len(flat_data[0])

In [ ]:
import seaborn as sns

unique,counts = np.unique(target,return_counts=True)

plt.figure(figsize=(9,8))
ax = sns.barplot(x=CATEGORIES, y=counts, linewidth=2.5, edgecolor="0.2")

for i, count in enumerate(counts):
    ax.text(i, count + 0.1, str(count), ha='center', va='bottom')
plt.show()

In [ ]:
# Split Data into Training and Testing

from sklearn.model_selection import train_test_split

xtrain,xtest,ytrain,ytest = train_test_split(flat_data,target,test_size=0.3, random_state=1)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = [
    {"C": [1, 10, 100, 1000], "kernel": ['linear']},
    {"C": [1, 10, 100, 1000], "gamma": [0.01, 0.001, 0.0001], "kernel": ['rbf']},
    {"degree": [2, 3, 4], "coef0": [0.0, 1.0, 2.0], "kernel": ['poly']}
]


svc = SVC(probability=True)
clf = GridSearchCV(svc,param_grid)
clf.fit(xtrain,ytrain)

In [ ]:
ypred = clf.predict(xtest)

In [ ]:
ypred

In [ ]:
ytest

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

accuracy_score(ypred, ytest)

In [ ]:
confusion_matrix(ypred,ytest)

In [ ]:
import pickle

pickle.dump(clf,open('img_model.p','wb'))

In [ ]:
model = pickle.load(open('img_model.p','rb'))